# FEB-II: basic operation

Referring to https://github.com/Mu2e/CRV_FEB2/blob/main/README.md 

In [1]:
# External imports 
import sys
# Internal imports
sys.path.append("..")
import CRV 

In [2]:
# Initialise CRV class
crv = CRV.CRV("/dev/ttyUSB0", True) # 2nd ROC is USB0

## Reset

```
0x000 	Main Control Register, 16 bits R/W (note: these bits are NOT self clearing!)
	bits 15..9, reserved
	bit 8 = clear DDR FIFO nearly full warning
	bit 7 = clear encoded FM clock parity error
	bit 6 = hard reset AFE chips
	bit 5 = general soft reset for FPGA logic (does not stop clocks)
	bit 2 = reset AFE front end (force front end recalibration and realignment)
	bit 1 = AFE1 power down
	bit 0 = AFE0 power down
```


In [3]:
crv.read("000", lc=True) # Check control register 
crv.write("000", "0020", lc=True) # Set bit 5 for soft reset 
crv.write("000", "0000", lc=True) # Clear bits

send:  LC RD 000
readback:  b'LC RD 000\r\r\n'
read : b'0000\r\n'
send:  LC WR 000 0020
readback:  b'LC WR 000 0020\r\r\n'
send:  LC WR 000 0000
readback:  b'LC WR 000 0000\r\r\n'


## Enable self-trigger mode

```
0x303 	Trigger Control Register
	Bit 0 = Self Trigger Enable (SlfTrgEn)
		0 = use FM encoded clock from ROC to generate timing signals (default)
		1 = use local free running counters to generate "fake" timing signals
	Bit 1 = VCXO 160MHz clock control ("TrgSrc") 
		0 = frequency lock the 160MHz clock to the ROC clock (default)
		1 = disable VCXO frequency locking to the ROC clock and 
		    force the VCXO control voltage to stable mid-point.	

	The fake timing signals are a crude approximation of a super cycle and 
	consist of 25k "on spill" windows of 1700ns followed by 10k "off spill"
	windows of 100us, repeats every 1.0425 seconds.
```

In [4]:
# Enable self-trigger mode
crv.write("303", "0001", lc=True)  # Broadcast write

send:  LC WR 303 0001
readback:  b'LC WR 303 0001\r\r\n'


## Map channels to test pulse 

```
0x080-0x08F Input mux control registers map physical channels to logical channels

0x080 Logical channel  0 mux control (default value = 0)
...
0x08F Logical channel 15 mux control (default value = 15)

	Each of these 5 bit registers is R/W and 
	the register contents are defined as:

	0 = AFE0 ch 0		8  = AFE1 ch 0
...
	7 = AFE0 ch 7		15 = AFE1 ch 7

	16 = fake positive going pulse aligned with start of livegap
	     this pulse starts at 0, sharp rise up to 0xAAA, then decay 
	     back to zero. for details see pulser.vhd
```

In [5]:
# crv.write("80", "0010", lc=True)  # Value 16 = test pulse

# Map all logical channels to the test pulse (value 0x0010)
for i in range(16):
    reg = f"0{i:X}" if i < 10 else f"{i:X}"  # Format: 00, 01, ..., 0F
    crv.write(f"8{reg}", "0010", lc=True)

send:  LC WR 800 0010
readback:  b'LC WR 800 0010\r\r\n'
send:  LC WR 801 0010
readback:  b'LC WR 801 0010\r\r\n'
send:  LC WR 802 0010
readback:  b'LC WR 802 0010\r\r\n'
send:  LC WR 803 0010
readback:  b'LC WR 803 0010\r\r\n'
send:  LC WR 804 0010
readback:  b'LC WR 804 0010\r\r\n'
send:  LC WR 805 0010
readback:  b'LC WR 805 0010\r\r\n'
send:  LC WR 806 0010
readback:  b'LC WR 806 0010\r\r\n'
send:  LC WR 807 0010
readback:  b'LC WR 807 0010\r\r\n'
send:  LC WR 808 0010
readback:  b'LC WR 808 0010\r\r\n'
send:  LC WR 809 0010
readback:  b'LC WR 809 0010\r\r\n'
send:  LC WR 8A 0010
readback:  b'LC WR 8A 0010\r\r\n'
send:  LC WR 8B 0010
readback:  b'LC WR 8B 0010\r\r\n'
send:  LC WR 8C 0010
readback:  b'LC WR 8C 0010\r\r\n'
send:  LC WR 8D 0010
readback:  b'LC WR 8D 0010\r\r\n'
send:  LC WR 8E 0010
readback:  b'LC WR 8E 0010\r\r\n'
send:  LC WR 8F 0010
readback:  b'LC WR 8F 0010\r\r\n'


## Check output FIFO 
```
0x017: 	Core Output FIFO Status 
	(note this register replaces the multi-FPGA broadcast read 0x317) R/O
	bit 4: output FIFO empty flag 
	bit 0: always 1
```

In [6]:
crv.read("017", lc=True)

send:  LC RD 017
readback:  b'LC RD 017\r\r\n'
read : b'0001\r\n'


['0001']

## Request data 

```
0x312 	Request UB number, HIGH order bits 31..16

0x313 	Request UB number, LOW order bits 15..0
	Writing to this register will initiate a read cycle from the 
	DDR memory controller and a short time later the requested event
	data will appear in the core output FIFO.

...

The microcontroller requests an event to be read from the DDR memory by writing the UB number to regs 0x312 and 0x313. 
(Always write 0x312 FIRST, as writing to 0x313 is the trigger to GO!) 
```

In [7]:
# Request UB=0
crv.read("312", "0000", lc=True)  # Upper bits
crv.write("313", "0000", lc=True)  # Lower bits (triggers read)
crv.read("017", lc=True) # Check output FIFO status again

send:  LC RDI 312 0000
readback:  b'LC RDI 312 0000\r\r\n'
read : b''
send:  LC WR 313 0000
readback:  b'LC WR 313 0000\r\r\n'
send:  LC RD 017
readback:  b'LC RD 017\r\r\n'
read : b'0001\r\n'


['0001']

## Read all data from FIFO 

Keep reading words until 0x017 = 011

In [8]:
crv.read("017", lc=True)

send:  LC RD 017
readback:  b'LC RD 017\r\r\n'
read : b'0001\r\n'


['0001']

In [9]:
# Temporarily disable verbose output
crv.verbose = False

# Read all data from FIFO
fifo_data = []
while crv.read("017", lc=True) != ['0011']:
    read_result = crv.read("00C", lc=True)
    if read_result:  # Check if the list is not empty
        fifo_data.append(read_result[0])
    else:
        break  # Exit the loop if we get an empty result

# Restore the original verbose setting
crv.verbose = True

print(f"Read {len(fifo_data)} words from FIFO")
# print("Data:", fifo_data)

Read 456 words from FIFO


In [10]:
def reshape_data(data, columns=8):
    """Reshape the data into rows of 'columns' values each."""
    reshaped = []
    for i in range(0, len(data), columns):
        if i+columns <= len(data):
            reshaped.append(data[i:i+columns])
    return reshaped

# Column headers for event builders
headers = ["EB0", "EB1", "EB2", "EB3", "EB4", "EB5", "EB6", "EB7"]

# Create header line
header_line = "     " + " ".join(f"{h:>5}" for h in headers)
print(header_line)
print("     " + "-" * (5*8 + 7))  # Separator line

# Reshape the data
reshaped_data = reshape_data(fifo_data)

# Display in a more readable format with column names
for i, row in enumerate(reshaped_data):
    print(f"{i:3d}: {' '.join(f'{val:>5}' for val in row)}")

       EB0   EB1   EB2   EB3   EB4   EB5   EB6   EB7
     -----------------------------------------------
  0:  000A  000A  000A  000A  000A  000A  000A  000A
  1:  0000  0002  0004  0006  0008  000A  000C  000E
  2:  0012  0012  0032  0012  0012  0012  0012  0012
  3:  5FAA  1FAE  8010  1FDF  7FE4  7FC8  9FC8  0FF0
  4:  ACFA  ACFB  FF00  E7FE  E3FE  C9FC  CBFC  E9FF
  5:  FB4F  FAAF  003F  FE6F  FE5F  FCBF  FCBF  FE4F
  6:  5FB5  BFAB  0008  5FDD  BFE2  BFCB  EFCE  EFDD
  7:  B3FB  AEFA  1401  D2FD  D7FD  C9FC  CAFC  E4FD
  8:  FB2F  FAFF  0170  FD3F  FDDF  FCDF  FCCF  FE8F
  9:  FFB5  1FB2  E024  AFDB  FFE6  1FD3  1FCF  1FF0
 10:  C0FB  ADFB  3A02  DDFD  ECFE  C9FD  D2FD  ECFF
 11:  FC0F  FADF  0430  FD0F  FE7F  FC9F  FD8F  FE2F
 12:  0000  0002  0004  0006  0008  000A  000C  000E
 13:  002A  002A  00D0  002A  002A  002A  002A  002A
 14:  AFC5  EFAA  0005  DFCB  AFE4  2FCC  FFDF  BFE0
 15:  C6FC  B1FA  FB00  CEFC  EAFE  D4FD  E1FD  E0FD
 16:  FC7F  FB7F  002F  FCEF  FEAF  FDAF  FE3F

Nice that we get words. Not sure what they mean, yet? 

In [11]:
crv.read("017", lc=True) # why is not 011? don't understand that. 

send:  LC RD 017
readback:  b'LC RD 017\r\r\n'
read : b''


[]

In [13]:
# original_verbose